In [1]:
using Distributed
addprocs(44);

@everywhere using Pkg; @everywhere Pkg.activate("/mnt/bucket/people/briandd/Projects/pulse_input_DDM.jl")

In [2]:
using PyPlot, StatsBase, Pandas, DataFrames, Distributions

@everywhere using pulse_input_DDM
using PyPlot, Optim, ForwardDiff, LineSearches
using ROCAnalysis, Pandas
using DataFrames
using ImageFiltering
using LinearAlgebra

In [3]:
ntrials,dt = Int(5e4),1e-2

(50000, 0.01)

In [4]:
#parameters of the latent model
pz = DataFrames.DataFrame(generative = vcat(1e-6,10.,-0.5,40.,1.,0.8,0.02), 
    name = vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    fit = vcat(falses(1),trues(4),trues(2)),
    initial = vcat(1.,5.,-5,100.,2.,0.2,0.08));

#initialize any parameter that is not going to be fit to its generative value
pz[:initial][pz[:fit] .== false] = pz[:generative][pz[:fit] .== false];

In [5]:
#parameters for the choice observation
pd = DataFrames.DataFrame(generative = 0.1, name = "bias", fit = true, initial = 0.);

In [6]:
#generate some simulated clicks times and trial durations
data = pulse_input_DDM.sample_clicks(ntrials,dt);

In [7]:
#simulate choices from the model given the generative parameters
pulse_input_DDM.sampled_dataset!(data, pz[:generative], pd[:generative][1]; num_reps=1, rng=4);

In [8]:
#compute the likelihood of the data given the generative parameters
compute_LL(pz[:generative],pd[:generative][1],data)

-11794.359448028112

In [9]:
#find the ML parameters
pz[:final], pd[:final], opt_output, state = optimize_model(pz[:initial], pd[:initial][1], 
        pz[:fit], pd[:fit], data)

Iter     Function value   Gradient norm 
     0     2.732715e+04     5.231976e+03
     1     1.972505e+04     1.510253e+04
     2     1.506637e+04     3.192668e+04
     3     1.336507e+04     1.675301e+04
     4     1.260324e+04     2.423752e+03
     5     1.198576e+04     1.723058e+03
     6     1.197915e+04     1.210649e+03
     7     1.197543e+04     1.193942e+03
     8     1.193083e+04     3.243760e+03
     9     1.182682e+04     3.171940e+02
    10     1.181540e+04     9.849842e+02
    11     1.181090e+04     1.193126e+03
    12     1.180806e+04     1.054634e+03
    13     1.180414e+04     6.302063e+02
    14     1.180167e+04     4.171051e+02
    15     1.180049e+04     1.166525e+02
    16     1.179974e+04     4.568792e+01
    17     1.179944e+04     3.285826e+01
    18     1.179922e+04     4.473883e+01
    19     1.179912e+04     1.220136e+02
    20     1.179909e+04     1.372230e+02
    21     1.179909e+04     1.455621e+02
    22     1.179909e+04     1.449385e+02
    23     1.179

([1.0e-6, 10.0998, -0.0870964, 38.4873, 0.968198, 0.79014, 0.0194703], 0.15105080607191684, Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [1.0986122886681098,-0.10033534773107555, ...]
 * Minimizer: [2.091836513509706,-0.001741929857023188, ...]
 * Minimum: 1.178731e+04
 * Iterations: 66
 * Convergence: true
   * |x - x'| ≤ 1.0e-16: false 
     |x - x'| = 1.28e-12 
   * |f(x) - f(x')| ≤ 1.0e-16 |f(x)|: true
     |f(x) - f(x')| = 0.00e+00 |f(x)|
   * |g(x)| ≤ 1.0e-12: false 
     |g(x)| = 1.60e-04 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 108
 * Gradient Calls: 67, Optim.BFGSState{Array{Float64,1},Array{Float64,2},Float64,Array{Float64,1}}([2.09184, -0.00174193, 3.65033, -0.0323183, -0.235545, -3.93886, 0.151051], [2.09184, -0.00174193, 3.65033, -0.0323183, -0.235545, -3.93886, 0.151051], [-0.000145452, 0.000101397, -8.54051e-6, 4.69503e-5, -0.000160008, 6.43651e-6, 1.25043e-5], 11787.31

In [10]:
#compute the Hessian around the ML parameters
H = compute_Hessian(pz[:final],pd[:final][1],pz[:fit],pd[:fit],data)

7×7 Array{Float64,2}:
   1271.39    -16366.8        …    451.605     -124.05      15.2953 
 -16366.8          2.78833e5      3364.82      -751.482   -340.27   
   -558.235     4746.02          -1907.72       484.979     -8.73015
   -721.721     5703.16          -3444.77       869.665    -13.9545 
    451.605     3364.82          10279.0      -2464.4       -5.34376
   -124.05      -751.482      …  -2464.4        623.621      1.1799 
     15.2953    -340.27             -5.34376      1.1799   167.523  

In [11]:
pz[:CI_z_plus], pd[:CI_z_plus], pz[:CI_z_minus], pd[:CI_z_minus] = compute_CI(H, pz[:final], 
    pd[:final][1], pz[:fit], pd[:fit], data)

#identify which ML parameters have generative parameters within the CI 
pz[:within_bounds] = (pz[:CI_z_minus] .< pz[:generative]) .& (pz[:CI_z_plus] .> pz[:generative])
pd[:within_bounds] = (pd[:CI_z_minus] .< pd[:generative]) .& (pd[:CI_z_plus] .> pd[:generative]);

In [12]:
pz

,generative,name,fit,initial,final,CI_z_plus,CI_z_minus,within_bounds
,Float64,String,Bool,Float64,Float64,Float64,Float64,Bool
1,1.0e-6,σ_i,false,1.0e-6,1.0e-6,1.0e-6,1.0e-6,false
2,10.0,B,true,5.0,10.0998,11.6094,8.82728,true
3,-0.5,λ,true,-5.0,-0.0870964,0.339386,-0.513566,true
4,40.0,σ_a,true,100.0,38.4873,49.5629,29.8867,true
5,1.0,σ_s,true,2.0,0.968198,1.3435,0.697733,true
6,0.8,ϕ,true,0.2,0.79014,0.862142,0.724151,true
7,0.02,τ_ϕ,true,0.08,0.0194703,0.0301317,0.0125812,true


In [13]:
pd

,generative,name,fit,initial,final,CI_z_plus,CI_z_minus,within_bounds
,Float64,String,Bool,Float64,Float64,Float64,Float64,Bool
1,0.1,bias,true,0.0,0.151051,0.306445,-0.00434358,true
